# Relationship Analysis Agent 테스트 (Production Level)

관계 분석 에이전트 (Level 2) 테스트 노트북

## 역할: "Social Network Analyst" (소셜 네트워크 분석가)
- 캐릭터 간 관계 매핑
- 관계 유형 분류 (FRIENDLY, RIVAL, ENEMY, ...)
- 관계 변화 추적 (evolved_from)

> **Note**: 캐릭터 이름은 Character Agent의 이름과 정확히 매칭되어야 합니다.

---

### 💡 방향성 의미론 (Directional Semantics)

| 관계 유형 | 방향성 | 설명 |
|----------|--------|------|
| **BETRAYED** | 단방향 | source=배신자 → target=피해자 |
| **MENTOR** | 단방향 | source=스승 → target=제자 |
| FRIENDLY | 양방향 | 상호 우정 |
| RIVAL | 양방향 | 상호 경쟁 |
| ENEMY | 양방향 | 상호 적대 |
| FAMILY | 양방향 | 가족 관계 |
| ROMANTIC | 양방향 | 연인/호감 |

In [1]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [2]:
SAMPLE_STORY = """
서진과 이민호는 한때 가장 친한 친구였다. 어릴 때부터 함께 검술을 배우며 자랐다.
하지만 이민호가 마을을 배신한 후, 그들은 적이 되었다.

하나는 서진의 오랜 동료이자 마을의 치료사였다. 그녀는 항상 서진을 걱정했다.

박서연은 마을의 정탐꾼으로, 서진과 하나를 도와 정보를 수집했다. 
그녀는 하나를 언니처럼 따랐다.
"""

# 시뮬레이션: Character Agent에서 추출된 캐릭터 목록
EXTRACTED_CHARACTERS = [
    {"name": "서진", "role": "protagonist", "traits": ["brave", "skilled"]},
    {"name": "하나", "role": "supporting", "traits": ["wise", "calm"]},
    {"name": "이민호", "role": "antagonist", "traits": ["bitter", "angry"]},
    {"name": "박서연", "role": "supporting", "traits": ["agile", "loyal"]}
]

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/internal/callback",
            "extracted_characters": EXTRACTED_CHARACTERS,
            "extracted_events": [],
            "relationship_graph": {}, "consistency_report": {}, "retry_count": 0,
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

## 1. Relationship Analysis 실행

In [3]:
from app.agents.analysis.relationship import relationship_analysis_node

async def test_relationship_analysis():
    print("🔗 Relationship Analysis (Production Level) 테스트...")
    state = create_base_state()
    return await relationship_analysis_node(state)

result = run_async(test_relationship_analysis())
graph = result.get("relationship_graph", {})

print(f"\n✅ 분석 완료:")
print(f"   - relationships: {len(graph.get('relationships', []))}개")
print(f"   - neo4j_edges: {len(graph.get('neo4j_edges', []))}개 (Machine-Readable)")

🔗 Relationship Analysis (Production Level) 테스트...
[RELATIONSHIP] Available characters: ['서진', '하나', '이민호', '박서연']

✅ 분석 완료:
   - relationships: 5개
   - neo4j_edges: 5개 (Machine-Readable)


## 2. Relationships (Human-Readable)

In [4]:
relationships = graph.get("relationships", [])

print("="*70)
print("🔗 Relationships (캐릭터 간 관계)")
print("="*70)

for rel in relationships:
    source = rel.get('source', 'N/A')
    target = rel.get('target', 'N/A')
    rel_type = rel.get('relation_type', 'UNKNOWN')
    strength = rel.get('strength', 5)
    desc = rel.get('description', '')
    evolved = rel.get('evolved_from', '')
    bidirectional = rel.get('bidirectional', True)
    
    strength_bar = '█' * strength + '░' * (10 - strength)
    
    # 방향성에 따라 화살표 표시
    arrow = '↔' if bidirectional else '→'
    
    print(f"\n{source} {arrow} {target}")
    print(f"   유형: {rel_type}" + (f" (단방향)" if not bidirectional else " (양방향)"))
    if evolved:
        print(f"   진화: {evolved} → {rel_type}")
    print(f"   강도: [{strength_bar}] {strength}/10")
    if desc:
        print(f"   설명: {desc}")

🔗 Relationships (캐릭터 간 관계)

서진 ↔ 이민호
   유형: FRIENDLY (양방향)
   강도: [█████████░] 9/10
   설명: 어릴 때부터 가장 친한 친구였던 서진과 이민호

이민호 → 서진
   유형: BETRAYED (단방향)
   진화: FRIENDLY → BETRAYED
   강도: [█████████░] 9/10
   설명: 이민호가 마을을 배신하면서 서진과의 우정을 깨뜨림

서진 ↔ 하나
   유형: FRIENDLY (양방향)
   강도: [████████░░] 8/10
   설명: 서진과 하나는 오랜 동료이자 믿을 수 있는 친구

박서연 ↔ 서진
   유형: FRIENDLY (양방향)
   강도: [███████░░░] 7/10
   설명: 박서연은 서진을 도와 정보를 수집하는 등 협력관계

박서연 ↔ 하나
   유형: FRIENDLY (양방향)
   강도: [███████░░░] 7/10
   설명: 박서연은 하나를 언니처럼 따르는 관계


## 3. 방향성 검증 (Directional Semantics Check)

In [5]:
# 단방향이어야 하는 관계 유형
UNIDIRECTIONAL_TYPES = {'BETRAYED', 'MENTOR'}

print("="*70)
print("🧭 Directional Semantics Check (방향성 검증)")
print("="*70)

direction_errors = []
for rel in relationships:
    rel_type = rel.get('relation_type', '')
    bidirectional = rel.get('bidirectional', True)
    
    if rel_type in UNIDIRECTIONAL_TYPES and bidirectional:
        direction_errors.append(
            f"{rel.get('source')} → {rel.get('target')}: {rel_type}는 bidirectional=false여야 함"
        )

if direction_errors:
    print(f"\n⚠️ DIRECTION WARNINGS ({len(direction_errors)}):")
    for err in direction_errors:
        print(f"   - {err}")
else:
    print("\n✅ 모든 관계의 방향성이 올바릅니다!")

🧭 Directional Semantics Check (방향성 검증)

✅ 모든 관계의 방향성이 올바릅니다!


## 4. Neo4j-Ready JSON (Machine-Readable)

In [6]:
neo4j_edges = graph.get("neo4j_edges", [])

print("="*70)
print("🔗 Neo4j-Ready JSON (Graph Edge 형식)")
print("="*70)
print(json.dumps(neo4j_edges, ensure_ascii=False, indent=2))
print(f"\n📊 Total edges: {len(neo4j_edges)}")

🔗 Neo4j-Ready JSON (Graph Edge 형식)
[
  {
    "source": "서진",
    "target": "이민호",
    "relationship_type": "FRIENDLY",
    "attributes": {
      "strength": 9,
      "description": "어릴 때부터 가장 친한 친구였던 서진과 이민호",
      "bidirectional": true,
      "evolved_from": null
    }
  },
  {
    "source": "이민호",
    "target": "서진",
    "relationship_type": "BETRAYED",
    "attributes": {
      "strength": 9,
      "description": "이민호가 마을을 배신하면서 서진과의 우정을 깨뜨림",
      "bidirectional": false,
      "evolved_from": "FRIENDLY"
    }
  },
  {
    "source": "서진",
    "target": "하나",
    "relationship_type": "FRIENDLY",
    "attributes": {
      "strength": 8,
      "description": "서진과 하나는 오랜 동료이자 믿을 수 있는 친구",
      "bidirectional": true,
      "evolved_from": null
    }
  },
  {
    "source": "박서연",
    "target": "서진",
    "relationship_type": "FRIENDLY",
    "attributes": {
      "strength": 7,
      "description": "박서연은 서진을 도와 정보를 수집하는 등 협력관계",
      "bidirectional": true,
      "evolved_from": null
   

## 5. 참조 무결성 검사

In [7]:
valid_names = {c["name"] for c in EXTRACTED_CHARACTERS}

print("="*70)
print("🔍 Referential Integrity Check")
print("="*70)
print(f"Valid names from Character Agent: {valid_names}")

integrity_errors = []
for rel in relationships:
    source = rel.get('source')
    target = rel.get('target')
    if source and source not in valid_names:
        integrity_errors.append(f"Source '{source}' not in Character list")
    if target and target not in valid_names:
        integrity_errors.append(f"Target '{target}' not in Character list")

if integrity_errors:
    print(f"\n❌ INTEGRITY ERRORS ({len(integrity_errors)}):")
    for err in integrity_errors:
        print(f"   - {err}")
else:
    print("\n✅ All names match Character Agent output - Referential Integrity OK!")

🔍 Referential Integrity Check
Valid names from Character Agent: {'박서연', '이민호', '하나', '서진'}

✅ All names match Character Agent output - Referential Integrity OK!


## 6. Re-Analysis 테스트 (Conflict Feedback)

In [8]:
async def test_re_analysis():
    print("🔄 Re-analysis 테스트 (Conflict Feedback)...")
    state = create_base_state()
    state["retry_count"] = 1
    state["relationship_graph"] = {
        "relationships": [
            {"source": "서진", "target": "이민호", "relation_type": "FRIENDLY", 
             "strength": 7, "bidirectional": True}
        ]
    }
    state["consistency_report"] = {
        "conflicts": [
            {"type": "RELATIONSHIP_CONFLICT", 
             "description": "배신 후 FRIENDLY는 부적절함. 텍스트에 '배신' '적'이라는 표현이 있음. " 
                           "BETRAYED 관계로 수정 필요 (이민호가 서진을 배신)"}
        ]
    }
    return await relationship_analysis_node(state)

re_result = run_async(test_re_analysis())
re_graph = re_result.get("relationship_graph", {})

print("\n" + "="*70)
print("📝 Re-Analysis 결과")
print("="*70)
print(json.dumps(re_graph, ensure_ascii=False, indent=2))

🔄 Re-analysis 테스트 (Conflict Feedback)...
[RELATIONSHIP] Available characters: ['서진', '하나', '이민호', '박서연']
[RELATIONSHIP] Re-analyzing with 1 conflicts as feedback

📝 Re-Analysis 결과
{
  "relationships": [
    {
      "source": "서진",
      "target": "이민호",
      "relation_type": "FRIENDLY",
      "strength": 7,
      "bidirectional": true
    }
  ]
}


## 7. Spring Boot Callback 데이터

In [9]:
callback_data = {
    "relationship_analysis": {
        "relationships": relationships,
        "neo4j_edges": neo4j_edges
    }
}

print("="*70)
print("📤 Spring Boot Callback 데이터 구조")
print("="*70)
print(json.dumps(callback_data, ensure_ascii=False, indent=2))

📤 Spring Boot Callback 데이터 구조
{
  "relationship_analysis": {
    "relationships": [
      {
        "source": "서진",
        "target": "이민호",
        "relation_type": "FRIENDLY",
        "strength": 9,
        "description": "어릴 때부터 가장 친한 친구였던 서진과 이민호",
        "bidirectional": true,
        "evolved_from": null
      },
      {
        "source": "이민호",
        "target": "서진",
        "relation_type": "BETRAYED",
        "strength": 9,
        "description": "이민호가 마을을 배신하면서 서진과의 우정을 깨뜨림",
        "bidirectional": false,
        "evolved_from": "FRIENDLY"
      },
      {
        "source": "서진",
        "target": "하나",
        "relation_type": "FRIENDLY",
        "strength": 8,
        "description": "서진과 하나는 오랜 동료이자 믿을 수 있는 친구",
        "bidirectional": true,
        "evolved_from": null
      },
      {
        "source": "박서연",
        "target": "서진",
        "relation_type": "FRIENDLY",
        "strength": 7,
        "description": "박서연은 서진을 도와 정보를 수집하는 등 협력관계",
        "bidirectional